In [1]:
import pandas as pd
from transformers import pipeline

# Load the Biobert-based model for text classification
classifier = pipeline("zero-shot-classification", model="dmis-lab/biobert-base-cased-v1.2")

# Read the medical data from the CSV file
data = pd.read_csv("Book2.csv")

# Extract the sentences from the 'review' column
medical_sentences = data['review'].tolist()

# Define candidate labels/categories
candidate_labels = ["Adverse Effects", "Product Quality", "Medical Information"]

# Classify each sentence using the Biobert-based model
for sentence in medical_sentences:
    # Perform classification
    result = classifier(sentence, candidate_labels)
    predicted_label = result["labels"][0]
    
    print(f"Sentence: {sentence}\nPredicted category: {predicted_label}\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence: "It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"
Predicted category: Product Quality

Sentence: "My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."
Predicted category: Product Quality

Sentence: "I used to take another oral contraceptive, which had 21 pill cycle, and wa

TypeError: object of type 'float' has no len()

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

# Load CSV data into DataFrame
df = pd.read_csv("book2.csv")

# Preprocess the data
# (You may need to perform additional preprocessing steps here)
text_data = df['review'].values
labels = df['Category'].values


# Split the data into training, validation, and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = BertTokenizer.from_pretrained('dmis-lab/biobert-v1.1')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

model = TFBertForSequenceClassification.from_pretrained('dmis-lab/biobert-v1.1', num_labels=len(label_encoder.classes_))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])

# Train the model
history = model.fit(x=train_encodings, y=train_labels, 
                    validation_data=(val_encodings, val_labels), 
                    batch_size=16, 
                    epochs=3)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_encodings, test_labels, batch_size=16)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

In [ ]:
!pip install tensorflow

In [7]:
print(type(train_texts))
print(type(val_texts))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
